# CAPSTONE PROJECT

## Environment study

Importing module (The need of importing the GLU module of OpenGL is a known bug in Roboschool [Issue 15](https://github.com/openai/roboschool/issues/15)). 

In [1]:
from OpenGL import GLU
import gym, roboschool

In [2]:
env = gym.make("RoboschoolHumanoid-v1")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'roboschool.gym_mujoco_walkers.RoboschoolHumanoid'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


### Action space

In [23]:
print("Class:", env.action_space)
print("Low:", env.action_space.low)
print("High:", env.action_space.high)

Class: Box(17,)
Low: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
High: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


The action space is a vector of 17 float values in the range [-1, 1]. Each value corresponds to the joints of the avatar by this order ([XML](https://github.com/openai/roboschool/blob/master/roboschool/mujoco_assets/humanoid_symmetric.xml)):
- abdomen_y
- abdomen_z
- abdomen_x
- right_hip_x
- right_hip_z
- right_hip_y
- right_knee
- left_hip_x
- left_hip_z
- left_hip_y
- left_knee
- right_shoulder1
- right_shoulder2
- right_elbow
- left_shoulder1
- left_shoulder2
- left_elbow
    
At each step, these values are applied to all the joints of the body by the code

```
for n,j in enumerate(self.ordered_joints):
    j.set_motor_torque( self.power*j.power_coef*float(np.clip(a[n], -1, +1)) )
```

in the `apply_action` function in the class which extends the `gym.Env` class (`RoboschoolMujocoXmlEnv`) to set the torque value into the respective motor.

Example:

In [28]:
print(env.action_space.sample())

[ 0.6652397   0.5563135   0.74002427  0.9572367   0.59831715 -0.07704128
  0.56105834 -0.76345116  0.27984205 -0.71329343  0.88933784  0.04369664
 -0.17067613 -0.47088876  0.5484674  -0.08769934  0.1368679 ]


### Observation space

In [29]:
print("Class:", env.observation_space)
print("Low:", env.observation_space.low)
print("High:", env.observation_space.high)

Class: Box(44,)
Low: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf]
High: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf]


The state space (or observation space) is a vector of 44 float values in the range [-5, 5] (Roboschool clip the vector with numpy before returning it in the `step` function). That vector is a concatenation of three subvectors:
- **more**: It is a vector of 8 values defined as follows:
    - The distance between the last position of the body and the current one.
    - The sinus of the angle to the target
    - The cosinus of the angle to the target
    - The three next values is the matrix multiplication between 
        - $\begin{pmatrix}cos(-yaw) & -sin(-yaw) & 0\\sin(-yaw) & cos(-yaw) & 0 \\0 & 0 & 1\end{pmatrix}$
        - the speed vector of the body.
    - The roll value of the body
    - The pitch value of the body
- **j** : This is the current relative position of the joint described earlier and their current speed. The position is in the even position, and the speed in the odds (34 values).
- **feet_contact**: Boolean values, 0 or 1, for left and right feet, indicating if the respective feet is touching the ground or not.
    
Example:

In [37]:
print(env.step(env.action_space.sample()))

(array([ 5.8925927e-01, -2.3844224e-01,  9.7115666e-01,  2.4020243e-01,
        7.3771449e-03, -2.5622851e-01, -1.0154423e-02, -1.0572182e-01,
       -1.4167477e-01, -4.7701159e-01,  7.2386289e-01,  1.8244406e+00,
       -1.7622218e-01, -3.1583855e-01,  7.8179151e-01, -7.5603100e-03,
        4.3423808e-01,  2.1802834e-01,  4.6744096e-01, -1.7865072e+00,
        9.6766943e-01, -9.6270740e-01,  2.5829774e-01, -4.5371920e-01,
        3.9913931e-01,  1.7034637e-02,  3.4505320e-01, -2.6078725e+00,
        8.4465736e-01, -2.4145675e+00,  1.1195302e-01, -6.8632201e-03,
       -1.3615746e-03, -5.7392287e-01,  7.0443526e-02, -1.7875153e+00,
       -1.9023059e-01, -1.8422981e-01, -2.0428559e-01, -1.9742851e-01,
        2.1261950e-01, -1.2376125e+00,  0.0000000e+00,  0.0000000e+00],
      dtype=float32), -3.1107794432085942, False, {})


### Reward

The reward is a sum of 5 computed values:

- **alive**: -1 or +1 wether is on the ground or not.
- **progress**: potential minus the old potential. The potential is defined by
    the speed multiplied by the distance to target point, to the negative.
- **electricity_cost**: The amount of energy needed for the last action.
- **joints_at_limit_cost**: The amount of collision between joints of body during the last action.
- **feet_colision_cost**: The amount of feet collision taken during the last action.